In [1]:
import numpy as np
import keras
from keras import layers
from keras import Input
from keras.models import Model
import tensorflow as tf

Using TensorFlow backend.


In [18]:
train_x = np.array([[103, 5, 2], #100
           [101, 5, 2], #100
           [140, 5, 2], #100
           [720, 6, 1], #721
           [130, 6, 1], #721
           [200, 41,3], #105
           [300, 5, 2], #100
           [470, 61, 1], #99
           [65, 44, 3], #102
           [98, 14, 2]]) #89

In [3]:
train_y = np.array([[100],
            [100],
            [100],
            [721],
            [721],
            [105],
            [100],
            [99],
            [102],
            [89]])

In [4]:
test_x = [[104, 5, 2], #100
           [199, 7, 3], #102
           [255, 41, 3], #105
           [625, 16, 1], #721
           [147, 14, 2], #89
           [456, 5, 2], #100
           [124, 5, 2], #100
           [789, 6, 1], #721
           [65, 61, 1], #99
           [11, 6, 1]] #721

In [5]:
test_y = [[100],
            [102],
            [105],
            [721],
            [89],
            [100],
            [100],
            [721],
            [99],
            [721]]

In [6]:
def get_three_inputs(oper_param_value):
    oper = []
    oper_param = []
    param_value = []
    for row in range (len(oper_param_value)):
        for value in range (len(oper_param_value[row])):
            oper.append(oper_param_value[row][:1])
            oper_param.append(oper_param_value[row][1:2])
            param_value.append(oper_param_value[row][2:])
            break
    return oper, oper_param, param_value

In [7]:
oper, oper_param, param_value = get_three_inputs(train_x)

In [8]:
oper_test, oper_param_test, param_value_test = get_three_inputs(test_x)

In [9]:
oper = np.array(oper)
oper_param = np.array(oper_param)
param_value = np.array(param_value)

In [10]:
# from keras.utils import to_categorical
# param_value = to_categorical(param_value, num_classes = 3)

In [11]:
# param_value = param_value.reshape(1, 10,1)

In [12]:
param_value = np.expand_dims(param_value, axis=2)
# param_value.ndim

In [13]:
oper_vocabulary_size = 10
param_vocabulary_size = 10
param_val_vocabulary_size = 10

In [14]:
from keras.models import Model
from keras import layers
from keras import Input

oper_input = keras.Input(shape=(None,), name = 'oper_input')
param_input = keras.Input(shape=(None,), name = 'param_input')
param_value_input = keras.Input(shape=(None, 10), name = 'param_value_input')

# Преобразование входного текста в последовательность векторов с размером 64
embedded_oper = layers.Embedding(oper_vocabulary_size, 64)(oper_input)
embedded_param = layers.Embedding(param_vocabulary_size, 64)(param_input)

concatenated = layers.concatenate([embedded_oper, embedded_param, param_value_input])

x_lstm1 = layers.LSTM(32, return_sequences=True)(concatenated)
x_lstm2 = layers.LSTM(32, return_sequences=False)(x_lstm1)

x_dense = layers.Dense(10, activation="relu") (x_lstm2)

# Добавление классификатора softmax сверху
predict = layers.Dense(oper_vocabulary_size, activation='softmax')(x_dense)

# Создание экземпляра модели с тремя входами и одним выходом
model = Model([oper_input, param_input, param_value_input], predict)

model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['acc'])

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
oper_input (InputLayer)         (None, None)         0                                            
__________________________________________________________________________________________________
param_input (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640         oper_input[0][0]                 
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     640         param_input[0][0]                
____________________________________________________________________________________________

In [16]:
# Отображение слоев 
keras.utils.plot_model(model, 'multi_input.png', show_shapes=True)

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [17]:
# Передача списка входов
model.fit([oper, oper_param, param_value], train_y,
          epochs = 10, 
          validation_data = ([oper_test, oper_param_test, param_value_test], test_y))

ValueError: Error when checking input: expected param_value_input to have shape (None, 10) but got array with shape (1, 1)